In [1]:
import numpy as np
from scipy.sparse import coo_matrix
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector

from QLS.numpy_linear_solver import NumPyLinearSolver   # classical
from QLS.hhl import HHL              # quantum HHL
import time

In [2]:
# ----- user parameters ------------------------------------------------------
make_hermitian = True       # True → Hermitian SPD; False → general non-Hermitian
target_kappa   = 1e2        # desired condition number κ(A)
density        = 0.9        # fraction of nonzero entries (0<density≤1)
noise_level    = 1e1       # relative off-diag noise for SPD case

In [3]:
# ----- problem definition ---------------------------------------------------
NUM_WORK_QUBITS = 4                
DIM             = 2 ** NUM_WORK_QUBITS

In [4]:
# ----- helpers ---------------------------------------------------------------
def generate_sparse_spd(n, kappa, density, noise_level):
    """Hermitian SPD with log-spaced eigenvalues and random sparse off-diag noise."""
    # 1) log-spaced eigenvalues
    eigs = np.logspace(0, np.log10(kappa), n)
    # 2) diagonal entries
    rows = np.arange(n); cols = rows; data = eigs
    A = coo_matrix((data, (rows, cols)), shape=(n, n))
    # 3) add symmetric off-diagonal noise
    total = n*n
    nnz   = int(density*total)
    off   = max(nnz - n, 0)
    off  -= off % 2
    half  = off//2
    if half>0:
        i = np.random.randint(0,n,half*2)
        j = np.random.randint(0,n,half*2)
        mask = (i!=j)
        i,j = i[mask][:half], j[mask][:half]
        eps  = noise_level * eigs.min()
        vals = np.random.uniform(-eps, eps, size=half)
        A   = A + coo_matrix((vals,(i,j)),shape=(n,n))\
                + coo_matrix((vals,(j,i)),shape=(n,n))
    return A.tocsr()

In [5]:
def generate_general(n, kappa, density):
    """General (non-Hermitian) matrix with approx κ via SVD, then sparsified."""
    # 1) U, V random orthonormal
    U,_  = np.linalg.qr(np.random.randn(n,n))
    V,_  = np.linalg.qr(np.random.randn(n,n))
    # 2) singular values
    s    = np.logspace(0, np.log10(kappa), n)
    A    = U @ np.diag(s) @ V.T
    # 3) sparsify by zeroing random entries
    mask = (np.random.rand(n,n) < density)
    A   *= mask
    return A

In [6]:
# ----- build A ----------------------------------------------------------------
if make_hermitian:
    A_sparse = generate_sparse_spd(DIM, target_kappa, density, noise_level)
    A        = A_sparse.toarray()
else:
    A = generate_general(DIM, target_kappa, density)


In [ ]:
# ----- checks & print ---------------------------------------------------------
is_herm = np.allclose(A, A.conj().T, atol=1e-12)
cond_A  = np.linalg.cond(A)
nnz     = np.count_nonzero(A)

print("A =\n", A)   # uncomment to see the full matrix

print(f"A (dim={DIM}×{DIM}), Hermitian? {is_herm}, κ(A) ≈ {cond_A:.3e}, "
      f"sparsity={nnz}/{DIM*DIM}={nnz/(DIM*DIM):.2%}\n")

A (dim=16×16), Hermitian? True, κ(A) ≈ 1.266e+02, sparsity=138/256=53.91%

A =
 [[  1.         -11.72022456   0.          16.96984509 -11.29285126
    0.           0.           0.           8.84608385   0.
    0.           5.23111979   4.16024239   0.           0.
    0.        ]
 [-11.72022456   1.35935639 -10.14480525   0.36111963   0.
    0.41895499   9.88610862   0.           7.7852984    0.
    0.          -0.47735863   4.92367501  -8.3793611   -0.1537671
    0.        ]
 [  0.         -10.14480525   1.8478498    7.20437412   0.
    0.           0.           0.          11.01708552   0.
   -9.69809648  -4.75776682 -17.3762155   -2.64921694   0.4597871
    9.47856873]
 [ 16.96984509   0.36111963   7.20437412   2.51188643  -9.43899037
   -0.48725316  -6.75937293   7.11980885   0.           6.58664558
   -6.93969511   0.           0.          13.37403773   9.08243056
   -0.95073336]
 [-11.29285126   0.           0.          -9.43899037   3.41454887
    0.           0.           5.287

In [8]:
# ----- right-hand side -------------------------------------------------------
b_vec = np.zeros(DIM, dtype=complex if np.iscomplexobj(A) else float)
b_vec[0] = 1
b_vec

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [9]:
# ----- classical solution ----------------------------------------------------
classical_res = NumPyLinearSolver().solve(
    A,
    b_vec / np.linalg.norm(b_vec)
)

In [10]:
# ----- quantum (HHL) solution ------------------------------------------------
# calculate the time for the quantum solver
start_time = time.time()


backend = AerSimulator(method='statevector',
                           device='GPU',
                           precision='double') 
print(backend.available_devices())
hhl_solver  = HHL(epsilon=1e-3, quantum_instance=backend)
quantum_res = hhl_solver.solve(A, b_vec)

def extract_solution(result, n_work_qubits: int) -> np.ndarray:
    sv           = Statevector(result.state).data
    total_qubits = int(np.log2(len(sv)))
    base_index   = 1 << (total_qubits - 1)
    amps         = np.array([sv[base_index + i]
                             for i in range(2 ** n_work_qubits)])
    return result.euclidean_norm * amps / np.linalg.norm(amps)

x_classical = classical_res.state
x_quantum   = extract_solution(quantum_res, NUM_WORK_QUBITS)
# calculate the time taken by the quantum solver
end_time = time.time()
# print time taken
print(f"Quantum solver took {end_time - start_time:.3f} seconds.")

['GPU']


Quantum solver took 102.994 seconds.


In [11]:
# ----- results ---------------------------------------------------------------
print("Classical solution vector:", x_classical)
print("Quantum   solution vector:", x_quantum, "\n")
print("Classical Euclidean norm:", classical_res.euclidean_norm)
print("Quantum   Euclidean norm:", quantum_res.euclidean_norm, "\n")
print("‖x_classical − x_quantum‖₂ =",
      np.linalg.norm(x_classical - x_quantum))

Classical solution vector: [-0.01175231 -0.06465738  0.05044731  0.00566854 -0.04168877  0.03870139
  0.0492514   0.02413412 -0.0344871  -0.02614336  0.00541299 -0.02174911
  0.02543649 -0.01065493 -0.00229392 -0.00487528]
Quantum   solution vector: [-0.01212654+1.52697098e-15j -0.06479186+2.53917008e-13j
  0.05113015-1.71171246e-13j  0.00607188+2.67358031e-14j
 -0.04198239+1.14036836e-13j  0.039251  -1.23031844e-13j
  0.04934294-2.02859643e-13j  0.02421617-8.28458694e-14j
 -0.03491803+1.22375016e-13j -0.02627768+8.18526078e-14j
  0.00562928-5.24464563e-15j -0.02201645+6.94320487e-14j
  0.0257156 -9.56460140e-14j -0.01077203+3.06955913e-14j
 -0.00235841+1.76583934e-15j -0.00492176+1.88066268e-14j] 

Classical Euclidean norm: 0.12768867420906418
Quantum   Euclidean norm: 0.1286616593186827 

‖x_classical − x_quantum‖₂ = 0.0012690205714452148


In [12]:
# %% [markdown]
# ## Log run results to JSON (robust to malformed file)

# %%
import os
import json
from datetime import datetime

def serialize_complex_vector(vec):
    return [[float(c.real), float(c.imag)] for c in vec]

record = {
    "timestamp":             datetime.now().isoformat(),
    "dim":                   DIM,
    "make_hermitian":        make_hermitian,
    "is_hermitian":          bool(is_herm),
    "condition_number":      cond_A,
    "nnz":                   nnz,
    "density":               nnz/(DIM*DIM),
    "noise_level":           noise_level if make_hermitian else None,
    "time_quantum_sec":      end_time - start_time,
    "euclid_norm_classical": classical_res.euclidean_norm,
    "euclid_norm_quantum":   quantum_res.euclidean_norm,
    "diff_norm":             float(np.linalg.norm(x_classical - x_quantum)),
    "x_classical":           serialize_complex_vector(x_classical),
    "x_quantum":             serialize_complex_vector(x_quantum),
    "matrix":                A.tolist(),
}

logfile = "hhl_runs_log.json"

# Try to load existing data; if it fails or is malformed, overwrite
try:
    with open(logfile, "r") as f:
        data = json.load(f)
    if not isinstance(data, list):
        raise ValueError("Top-level JSON is not a list")
except (FileNotFoundError, json.JSONDecodeError, ValueError):
    data = []

data.append(record)

with open(logfile, "w") as f:
    json.dump(data, f, indent=2)

print(f"Logged run to {logfile} (total runs: {len(data)})")

Logged run to hhl_runs_log.json (total runs: 7)
